# MNIST Classification
> Performance analysis for MNIST Classification on all hardware platforms

- toc: true 
- badges: true
- comments: true
- categories: [MNIST,Rooflines,Performance Prediction]
- image: images/mnist.png

In [1]:
#hide
import pandas as pd
import numpy as np
import altair as alt

W = 600
H = 480

pd.options.display.float_format = '{:20,.7f}'.format
pd.options.display.max_rows =100000 # this will set limit of columns to 500
pd.options.display.max_columns =100000 # this will set limit of columns to 500
#master_df.loc[master_df['NN_Topology'] =='MLP']

csv_path = "./data/cleaned_csv/backup.csv"

# Theoretical Analysis of MNIST

### Rooflines for All Hardware Platforms and CNNs

Combining application requirements with hardware platform characteristics can be leveraged for performance predictions using UCB’s roofline models. Using assumptions for where weights, activation tensors, and state of a neural network are stored, combined with the size of the datatypes used, allow us to derive the arithmetic intensity of a neural network during inference. Combined with the roofline for a given hardware platform, we can provide insight as to whether a neural network will be memory or compute bound and guidance for what is theoretically possible in regards to its throughput.

In [2]:
#hide_input
# Run the Rooflines script which processes the data and creates the chart
%run scripts/altair_plots.py
rooflines('mnist')

alt.LayerChart(...)

*Applies to the following pruning factors: 100%, 50%, 25% and 12,5% 

### Performance Prediction

The following heatmap shows the theoretical performance for the listed hardware platforms for MNIST classification. The metric used for the theoretical performance is input/second.
This plot shows that a combination between prunning and quantization outputs some of the best performance results.

In [3]:
#hide_input
%run scripts/altair_plots.py
heatmap('data/performance_predictions_imagenet_mnist_cifar.csv', 'mnist', 'Performance Prediction for MNIST')

alt.LayerChart(...)

# Experimental Data Analysis

### Overview of All Measurements for MNIST 

In this table, within the rows, we show the type of hardware platforms that we used for this task (for example FPGA or GPU) and then more specifically the exact name of the different hardware platforms. For each hardware platform, we list the sweep of specific deployment parameters (batch sizes, operating modes etc) that were used for the experimentation in separate columns. In the columns, we show CNN topologies. When a CNN topology was implemented on a given hardware platform, we show in the corresponding cell the precisions (quantization information) and the channel pruning scale. Otherwise, “na” indicates that the topology wasn’t executed on this specific hardware platform. Many combinations between topology and hardware platform are not supported by the vendors dedicated software environments. INTx depicts a fixed point integer representation with x bits. FPy represents a floating point representation with y bits, for example FP32 is singe precision floating point. Table follows below.

In [4]:
#hide
%run scripts/script_tables.py  #get table with the experiments overview
print(pd.read_csv('data/overview_experiments_mnist.csv').to_markdown())

|    | Hardware   | Platform         | MLP                                 | Batch/Stream/Thread                  |
|---:|:-----------|:-----------------|:------------------------------------|:-------------------------------------|
|  0 | FPGA       | ZCU102-DPU       | na                                  | [1,2,3,4,5,6,7,8]                    |
|  1 | FPGA       | ZCU104-DPU       | na                                  | [1,2,3,4,5,6,7,8]                    |
|  2 | FPGA       | Ultra96-DPU      | na                                  | [1,2,3,4,5,6,7,8]                    |
|  3 | FPGA       | ZCU104-FINN      | [INT2, INT4] * [100%,50%,25%,12.5%] | [1,2,4,8,16,32,64,128,256,512,10000] |
|  4 | FPGA       | ZCU104-BISMO     | [INT2, INT4] * [100%,50%,25%,12.5%] | [2,4,8,16,32,64,128]                 |
|  5 | GPU        | TX2-maxn         | [FP16, FP32] * [100%,50%,25%,12.5%] | [1,2,4,8,16,32,64,128]               |
|  6 | GPU        | TX2-maxp         | [FP16, FP32] * [100%,50%,25%,12.5

In [5]:
#hide_input
%run scripts/script_tables.py 
dataframes = csv_to_dataframe_multiindex(['data/overview_experiments_mnist_.csv'])
for dataframe in dataframes:   
       display(HTML(dataframe.to_html(index=False)))

In [6]:
#hide
master_df = pd.read_csv(csv_path)
is_maxp = lambda row: row.HWType != "TX2" or row["Op mode"].split(",")[0] == "maxp"
maxp_df = master_df[master_df.apply(is_maxp, axis=1)]
maxp_df["hw_datatype_prun_net"] = maxp_df.apply(lambda r: "_".join([r.HWType, r.Datatype, r.PruningFactor, r.NN_Topology]), axis=1)
mnist_df = maxp_df[(maxp_df["NN_Topology"] == "MLP") & maxp_df['lat-comp'].notna() & maxp_df["top1 [%]"].notna()]
mnist_df["hw_datatype_prun_net"] = mnist_df.apply(lambda r: "_".join([r.HWType, r.Datatype, r.PruningFactor, r.NN_Topology]), axis=1)
mnist_df["PruningFactor"] = mnist_df["PruningFactor"].str.strip("%").astype(float)
norm_by_group(mnist_df, "lat-comp", "NN_Topology");
mnist_df["datatype_model"] = mnist_df.Datatype + '_' + mnist_df.HWType
#mnist_df= maxp_df[(maxp_df.NN_Topology == 'MLP') & maxp_df["top1 [%]"].notna()]
mnist_df.rename(columns={"top1 [%]": "top1"}, inplace=True)
mnist_df["tag"] = mnist_df.apply(lambda r: "_".join([r.HWType, r.Datatype, r.NN_Topology, str(r.PruningFactor)]), axis=1)

#filling GOPS values 
mnist_df['GOPS'] = mnist_df.apply(lambda r: 0.020029 if r.PruningFactor == 100 else 
                                          (0.00582 if r.PruningFactor == 50 else 
                                           (0.001862 if r.PruningFactor == 25 else 0.000669 )) , axis=1)
#fill in tp-system and tp-cmp
mnist_df['tp-system'] = mnist_df['GOPS'] * mnist_df['fps-system']
mnist_df['tp-comp'] = mnist_df['GOPS'] * mnist_df['fps-comp']
mnist_df['GOPS'] = mnist_df['GOPS'] * mnist_df['batch/thread/stream']
#mnist_df.head()

C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [7]:
#hide
mnist_df

,NN_Topology,HWType,Datatype,Op mode,batch/thread/stream,lat-sys,lat-comp,fps-system,fps-comp,tp-system,tp-comp,top1,top5 [%],Base_Pwr_W,Idle_Pwr_W,Full_Pwr_W,GOPS,PruningFactor,level,hw_peak_perf,hw_bandwidth,nn_total_operations,hw_datatype_prun_net,norm-lat-comp,datatype_model,tag
143,MLP,NCS,FP16,NaN,1,4.57,2.8925700,218.8930000,345.7130000,4.3842079,6.9242857,98.8600000,nan,0.5300000,1.2000000,1.7940000,0.0200290,100.0000000,l3,0.5000000,12.8000000,0.0200000,NCS_FP16_100%_MLP,0.0066456,FP16_NCS,NCS_FP16_MLP_100.0
144,MLP,NCS,FP16,NaN,2,8.09,4.6845900,247.3080000,426.9310000,4.9533319,8.5510010,98.8600000,nan,0.5300000,1.2000000,1.9490000,0.0400580,100.0000000,l3,0.5000000,12.8000000,0.0200000,NCS_FP16_100%_MLP,0.0107626,FP16_NCS,NCS_FP16_MLP_100.0
145,MLP,NCS,FP16,NaN,4,14.84,8.1182100,269.4910000,492.7190000,5.3976352,9.8686689,98.8600000,nan,0.5300000,1.2000000,1.9460000,0.0801160,100.0000000,l3,0.5000000,12.8000000,0.0200000,NCS_FP16_100%_MLP,0.0186512,FP16_NCS,NCS_FP16_MLP_100.0
146,MLP,NCS,FP16,NaN,8,28.3,14.8038000,282.6860000,540.4000000,5.6619179,10.8236716,98.8600000,nan,0.5300000,1.2000000,2.0840000,0.1602320,100.0000000,l3,0.5000000,12.8000000,0.0200000,NCS_FP16_100%_MLP,0.0340111,FP16_NCS,NCS_FP16_MLP_100.0
147,MLP,NCS,FP16,NaN,16,54.64,27.4935000,292.8480000,581.9560000,5.8654526,11.6559967,98.8600000,nan,0.5300000,1.2000000,2.1360000,0.3204640,100.0000000,l3,0.5000000,12.8000000,0.0200000,NCS_FP16_100%_MLP,0.0631651,FP16_NCS,NCS_FP16_MLP_100.0
148,MLP,NCS,FP16,NaN,32,107.39,53.0572000,297.9700000,603.1220000,5.9680411,12.0799305,98.8600000,nan,0.5300000,1.2000000,2.1350000,0.6409280,100.0000000,l3,0.5000000,12.8000000,0.0200000,NCS_FP16_100%_MLP,0.1218966,FP16_NCS,NCS_FP16_MLP_100.0
149,MLP,NCS,FP16,NaN,64,212.26,104.4350000,301.5140000,612.8230000,6.0390239,12.2742319,98.8600000,nan,0.5300000,1.2000000,2.1860000,1.2818560,100.0000000,l3,0.5000000,12.8000000,0.0200000,NCS_FP16_100%_MLP,0.2399348,FP16_NCS,NCS_FP16_MLP_100.0
150,MLP,NCS,FP16,NaN,128,421.84,206.0070000,303.4340000,621.3370000,6.0774796,12.4447588,98.8600000,nan,0.5300000,1.2000000,2.2120000,2.5637120,100.0000000,l3,0.5000000,12.8000000,0.0200000,NCS_FP16_100%_MLP,0.4732921,FP16_NCS,NCS_FP16_MLP_100.0
169,MLP,TX2,FP16,maxp,1,1.03,0.6982660,976.1680000,"1,466.4100000",19.5516689,29.3707259,97.3000000,nan,1.8000000,4.7000000,9.8810000,0.0200290,100.0000000,l3,1.1500000,59.7000000,0.0200000,TX2_FP16_100%_MLP,0.0016042,FP16_TX2,TX2_FP16_MLP_100.0
170,MLP,TX2,FP16,maxp,2,1.25,0.7399800,"1,635.7800000","2,785.5500000",32.7630376,55.7917810,97.3000000,nan,1.8000000,4.7000000,9.9030000,0.0400580,100.0000000,l3,1.1500000,59.7000000,0.0200000,TX2_FP16_100%_MLP,0.0017001,FP16_TX2,TX2_FP16_MLP_100.0


### Line Plot

In [8]:
#hide_input
figa_df = mnist_df[(mnist_df["HWType"].isin(["NCS", "ZCU104-Bismo", "U96-Quadcore A53"]))]
figb_df = mnist_df[(mnist_df["HWType"].isin(["TX2", "ZCU104-FINN", "U96-Quadcore A53"]))]

fig25s = []
fig25_dfs = [figa_df, figb_df]
for df in fig25_dfs:
    sel = alt.selection_multi(fields=["hw_datatype_prun_net"], bind="legend")
    fig25_dot = alt.Chart(df).mark_point().encode(
        x='lat-comp',
        y=alt.Y('fps-comp', scale=alt.Scale(type="log")),
        color=select_color(sel, 'hw_datatype_prun_net:N'),
        tooltip=['fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
    )
    fig25_line = alt.Chart(df).mark_line().encode(
        x='lat-comp',
        y='fps-comp',
        color=select_color(sel, 'hw_datatype_prun_net:N'),
        tooltip=['fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
    )

    fig = (fig25_dot+fig25_line).properties(
        title="Latency Versus Performance for Pruned and Quantized MLP Variants",
        width=W/len(fig25_dfs),
        height=H,
    ).add_selection(sel).interactive()
    
    fig25s.append(fig)
    
alt.hconcat(*fig25s)

alt.HConcatChart(...)

### Boxplots

In [9]:
#hide_input
%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=mnist_df, xaxis='PruningFactor', yaxis="lat-comp", color_col= 'PruningFactor', facet_column='datatype_model' , title="Latency by Hardware/Framework and Pruning for MNIST Classification")

alt.FacetChart(...)

In [10]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=mnist_df, xaxis='PruningFactor', yaxis="fps-comp", color_col= 'PruningFactor', facet_column='datatype_model' , title="Throughput by Hardware/Framework and Pruning for MNIST Classification")

alt.FacetChart(...)

In [11]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=mnist_df, xaxis='PruningFactor', yaxis="Full_Pwr_W", color_col= 'PruningFactor', facet_column='datatype_model' , title="Power Consumption by Hardware/Framework and Pruning for MNIST Classification")

alt.FacetChart(...)

### Pareto Graphs

The following pareto graph presents the accuracy versus performance in fps for all the Hardware Platforms across different Pruning and Quantization configurations. This provides insights into accuracy-based comparisons.

In [12]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
pareto_graph(df= mnist_df, 
             groupcol= 'hw_datatype_prun_net',
             xcol = 'fps-comp', 
             ycol= 'top1',
             W= W, 
             H= H, 
             title= "MNIST Cassification Design Space: Accuracy Versus Performance")

     NN_Topology            HWType Datatype Op mode  batch/thread/stream  \
143          MLP               NCS     FP16     NaN                    1   
144          MLP               NCS     FP16     NaN                    2   
145          MLP               NCS     FP16     NaN                    4   
146          MLP               NCS     FP16     NaN                    8   
147          MLP               NCS     FP16     NaN                   16   
148          MLP               NCS     FP16     NaN                   32   
149          MLP               NCS     FP16     NaN                   64   
150          MLP               NCS     FP16     NaN                  128   
169          MLP               TX2     FP16    maxp                    1   
170          MLP               TX2     FP16    maxp                    2   
171          MLP               TX2     FP16    maxp                    4   
172          MLP               TX2     FP16    maxp                    8   
173         

alt.LayerChart(...)

In [13]:
#hide
def replace_data_df(df_: pd.DataFrame(), column:str, list_tuples_data_to_replace: list )-> pd.DataFrame():
    """Method to replace a substring inside a cell inside a dataframe"""
    df = df_.copy()
    for j, k in list_tuples_data_to_replace:
        df[column] = df[column].str.replace(j, k)
    return df

In [14]:
#hide
""" This is to create a df to be joined with the theoretical df in 'Theoretical Analysis' to create the overlapped paretos

Steps
------
1. Create subset from imagenet that doesn't have the ResNet50 v15 measurements because it does not have accuracy measures
2. Create new hardware column that has hardware and operation mode, beware with NaNs
3. Create new 'hardw_datatype_net_prun' with hardware + datatype + netwrok + pruning
4. Create a suset of the dataframe with the above mentioned column and the corresponding ones
5. With groupby for col 'hardw_datatype_net_prun', for each unique value get the rows with biggest batch 
6. Add 'type column', reset the index from 'hardw_datatype_net_prun' to ints and save it
"""
#This is to get the pareto plot for biggest batch size
#Create mnist_df_tmp from mnist_df
mnist_df_tmp = mnist_df.copy()
#   create hardw column to include: hardware + op_mode
mnist_df_tmp['hardw']= mnist_df_tmp['HWType'] + ('-' + mnist_df_tmp['Op mode']).fillna('')
#   create hardw_datatype_net_prun col with all those columns merged
mnist_df_tmp['hardw_datatype_net_prun'] = mnist_df_tmp.apply(lambda r: "_".join([r.hardw, r.Datatype,r.NN_Topology, str(r.PruningFactor)]), axis=1)
#   create a subset of the dataframe with only those columns
mnist_df_tmp = mnist_df_tmp[['hardw_datatype_net_prun', 'hardw','NN_Topology','fps-comp', 'top1','batch/thread/stream']]
#   Only get the points corresponding to the biggest batch
mnist_df_tmp = mnist_df_tmp.groupby('hardw_datatype_net_prun')[['batch/thread/stream', 'hardw','NN_Topology','fps-comp', 'top1']].max()
#   add type column
mnist_df_tmp['type'] = 'measured'
#   reset index to start being numeric 
mnist_df_tmp = mnist_df_tmp.reset_index()
#   change column names
mnist_df_tmp.columns = ['hardw_datatype_net_prun', 'batch/thread/stream', 'hardw', 'network', 'fps-comp', 'top1', 'type']
#   fix samll stuff in the df so things match with the other side
mnist_df_tmp = replace_data_df(df_=mnist_df_tmp, column='hardw_datatype_net_prun', list_tuples_data_to_replace=[('.0','')])
#mnist_df_tmp = replace_data_df(df_=mnist_df_tmp, column='network', list_tuples_data_to_replace=[("RN50", "ResNet50"),("MNv1", "MobileNetv1"),('GNv1','GoogLeNetv1')])
#delete unnecessary columns
mnist_df_tmp = mnist_df_tmp.drop(columns=['batch/thread/stream'])

#save it all
mnist_df_tmp.to_csv('data/cleaned_csv/pareto_data_mnist.csv', index = False)
#plot the pareto to confirm all is good
pareto_graph(df= mnist_df_tmp, 
             groupcol= 'hardw_datatype_net_prun', 
             xcol= 'fps-comp', 
             ycol= 'top1', 
             W= W, 
             H= H, 
             title= "MNIST Cassification Design Space: Accuracy Versus Performance")
mnist_df_tmp.head() 

           hardw_datatype_net_prun             hardw network  \
0                   NCS_FP16_MLP_0               NCS     MLP   
1                NCS_FP16_MLP_12.5               NCS     MLP   
2                  NCS_FP16_MLP_25               NCS     MLP   
3                    NCS_FP16_MLP_               NCS     MLP   
4              TX2-maxp_FP16_MLP_0          TX2-maxp     MLP   
5           TX2-maxp_FP16_MLP_12.5          TX2-maxp     MLP   
6             TX2-maxp_FP16_MLP_25          TX2-maxp     MLP   
7               TX2-maxp_FP16_MLP_          TX2-maxp     MLP   
8              TX2-maxp_FP32_MLP_0          TX2-maxp     MLP   
9           TX2-maxp_FP32_MLP_12.5          TX2-maxp     MLP   
10            TX2-maxp_FP32_MLP_25          TX2-maxp     MLP   
11              TX2-maxp_FP32_MLP_          TX2-maxp     MLP   
12     U96-Quadcore A53_INT2_MLP_0  U96-Quadcore A53     MLP   
13  U96-Quadcore A53_INT2_MLP_12.5  U96-Quadcore A53     MLP   
14    U96-Quadcore A53_INT2_MLP_25  U96-

,hardw_datatype_net_prun,hardw,network,fps-comp,top1,type
0,NCS_FP16_MLP_0,NCS,MLP,621.3370000,98.8600000,measured
1,NCS_FP16_MLP_12.5,NCS,MLP,"2,796.1900000",97.9500000,measured
2,NCS_FP16_MLP_25,NCS,MLP,"2,197.4000000",98.4900000,measured
3,NCS_FP16_MLP_,NCS,MLP,"1,370.9600000",98.7500000,measured
4,TX2-maxp_FP16_MLP_0,TX2-maxp,MLP,"61,409.3000000",97.3000000,measured


# Theoretical Pareto and Measured Pareto Overlapped

In order to easily understand how accurate predictions were, an overlapping between the Theoretical Pareto Plot and Measured Pareto Plot was made. In the plot below we have both theoretical (orange) and measured (blue) pareto lines. All measured datapoins are represented as crosses and all theoretical datatpoins are represented as circles. Some theoretical datapoints don't have a measured matched datapoint and the same goes for the measured datapoints. The theoretical pareto curve is, as expected, on the right of the measured one, as predictions are sometimes different form measurements.

In [17]:
#hide_input
%run scripts/overlapped_pareto.py
get_overlapped_pareto('mnist','Overlapped Pareto Plots Theoretical + Measured for MNIST') 

alt.LayerChart(...)

# Efficiency Plot

In order to understand the gap between the theoretical predictions and what was measured, an efficiency bar-chart was created. The size of the bar reflects the absolute performance, whereby all theoretical predictions are shown in red, theoretical peak performance in blue, and all measured datapoints in orange. The orange bars are annotated with the efficiency achieved as a percentage of the predicted performance. Please note the logarithmic y-axis scale. The theoretical predictions take memory bottlenecks into account, as such measured performance can actually exceed the predicted result, in which case the percentage can be above 100%.

In [19]:
#hid
#%run scripts/overlapped_pareto.py
mnist_efficiency_df = get_peak_perf_gops_df(df_=mnist_df_tmp) #takes the mnist df and fills it with data for the 3rd bar - Theoretical Peak Performance
efficiency_plot(net_keyword= 'mnist', df_theo_peak_compute=mnist_efficiency_df ,title='Efficiency Plots for MNIST')

KeyError: "None of [Index(['hw_datatype_prun_net'], dtype='object')] are in the [columns]"

In [16]:
#hide
mnist_df.to_csv('data/cleaned_csv/experimental_data_mnist.csv', index = False)
mnist_df.head()

,NN_Topology,HWType,Datatype,Op mode,batch/thread/stream,lat-sys,lat-comp,fps-system,fps-comp,tp-system,tp-comp,top1,top5 [%],Base_Pwr_W,Idle_Pwr_W,Full_Pwr_W,GOPS,PruningFactor,level,hw_peak_perf,hw_bandwidth,nn_total_operations,hw_datatype_prun_net,norm-lat-comp,datatype_model,tag
143,MLP,NCS,FP16,NaN,1,4.57,2.8925700,218.8930000,345.7130000,4.3842079,6.9242857,98.8600000,nan,0.5300000,1.2000000,1.7940000,0.0200290,100.0000000,l3,0.5000000,12.8000000,0.0200000,NCS_FP16_100%_MLP,0.0066456,FP16_NCS,NCS_FP16_MLP_100.0
144,MLP,NCS,FP16,NaN,2,8.09,4.6845900,247.3080000,426.9310000,4.9533319,8.5510010,98.8600000,nan,0.5300000,1.2000000,1.9490000,0.0400580,100.0000000,l3,0.5000000,12.8000000,0.0200000,NCS_FP16_100%_MLP,0.0107626,FP16_NCS,NCS_FP16_MLP_100.0
145,MLP,NCS,FP16,NaN,4,14.84,8.1182100,269.4910000,492.7190000,5.3976352,9.8686689,98.8600000,nan,0.5300000,1.2000000,1.9460000,0.0801160,100.0000000,l3,0.5000000,12.8000000,0.0200000,NCS_FP16_100%_MLP,0.0186512,FP16_NCS,NCS_FP16_MLP_100.0
146,MLP,NCS,FP16,NaN,8,28.3,14.8038000,282.6860000,540.4000000,5.6619179,10.8236716,98.8600000,nan,0.5300000,1.2000000,2.0840000,0.1602320,100.0000000,l3,0.5000000,12.8000000,0.0200000,NCS_FP16_100%_MLP,0.0340111,FP16_NCS,NCS_FP16_MLP_100.0
147,MLP,NCS,FP16,NaN,16,54.64,27.4935000,292.8480000,581.9560000,5.8654526,11.6559967,98.8600000,nan,0.5300000,1.2000000,2.1360000,0.3204640,100.0000000,l3,0.5000000,12.8000000,0.0200000,NCS_FP16_100%_MLP,0.0631651,FP16_NCS,NCS_FP16_MLP_100.0
